In [18]:
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold
from random import sample
from scipy import stats
from sklearn.metrics import mean_absolute_error
from collections import defaultdict
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
#Model
folds = 10 #number of folds
features = 1 #number of features to predict
epochs = 180 #number of epochs
optimiser = 'adam' #model optimiser
loss = 'mse' #model loss
ntrees = 100

In [5]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [7]:
#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [29]:
os.chdir('/home/mxenoc/workspace/butterfly/src/butterfly')

In [31]:
os.chdir('/home/mxenoc/workspace/butterfly/notebooks')

In [9]:
with open('albums_all.pkl', 'rb') as f:
    albums_all = pickle.load(f)

In [10]:
with open('RF_predictor.pkl', 'rb') as f:
    RF_predictor = pickle.load(f)

In [11]:
with open('albums.pkl', 'rb') as f:
    albums = pickle.load(f)

In [32]:
with open('responses_30.pkl', 'rb') as f:
    responses = pickle.load(f)

In [ ]:
n_samples = 30 #same as the number of responses

CNN = defaultdict(list)
MCNN = defaultdict(list)
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.CNN)(albums_all, DF, feat_n, predictor_index, groups, 
                            responses, 128, features, folds, epochs, optimiser, loss, 'CNN')for feat_n in range(n_samples)))
        
    CNN['prediction_train'].append(prediction_train)
    CNN['observed_train'].append(observed_train)
    CNN['prediction_test'].append(prediction_test)
    CNN['observed_test'].append(observed_test)
    
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.CNN)(albums, DF, feat_n, predictor_index, groups, 
                            responses, 40, features, folds, epochs, optimiser, loss, 'MCNN')for feat_n in range(n_samples)))
        
    MCNN['prediction_test'].append(prediction_train)
    MCNN['observed_test'].append(observed_train)
    MCNN['prediction_test'].append(prediction_test)
    MCNN['observed_test'].append(observed_test)
    
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'RF_regressor')for feat_n in range(n_samples)))
        
    RF['prediction_test'].append(prediction_train)
    RF['observed_test'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)








  0%|          | 0/7 [00:00<?, ?it/s]